# Cyberbullying Text Classification
CS6120 Group 10: Pushyanth Damarapati, Sindhya Balasubramanian, Eileen Chang, Priyanka Padinam

### Description
The rise of social media and the recent couple of years of covid-19 lockdown has led to a concerning increase in cyberbullying cases. In 2020, UNICEF even issued a warning in response to the increased cyberbullying compounded by social distancing and increased screen-time. Those who bully others on the internet have the convenience of being able to hide anonymously behind a screen, but the people who are bullied are likely to develop mental-health issues that persist even after the bullying has ceased. Due to social media’s ability to spread information quickly and anonymously, a single person can easily end up being targeted by a large number of people of various demographics. We aim to create a model that will flag harmful tweets and, therefore, protect targets of cyberbullying.

### Dataset
We will be using a kaggle dataset, Cyberbullying Classification, consisting of more than 47,000 tweets labeled according to 6 classes of cyberbullying: Age, Ethnicity, Gender, Religion, Other type of cyberbullying, and Not cyberbullying. Each row of the dataset will have a tweet and its class of cyberbullying. The dataset is meant to be used to create a multi-classification model to predict cyberbullying type, create a binary classification model to flag potentially harmful tweets, and examine words and patterns associated with each type of cyberbullying.

# Importing Dataset and Libraries

In [1]:
!pip install contractions
# !pip install nltk
!pip install autocorrect 
# !pip install --upgrade matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 110 kB 15.5 MB/s 
     |████████████████████████████████| 287 kB 88.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 622 kB 12.0 MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622381 sha256=93befd8858e0e4a0103e6bb699241a3589e977d84d0c9a86a77e0278e3ddcc9a
  Stored in directory: /root/.cache/pip/wheels/72/b8/3b/a90246d13090e85394a8a44b78c8abf577c0766f29d6543c75
Successfully built autocorrect


In [2]:
import string 
import nltk 
import re # regex
from string import punctuation 
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer 
# from autocorrect import Speller # correct spelling
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

#Libraries 
import matplotlib.pyplot as plt
import seaborn as sns


#Data preprocessing
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
# from imblearn.over_sampling import RandomOverSampler

# #Naive Bayes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

import os

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
import numpy as np
import pandas as pd

df = pd.read_csv('cyberbullying_tweets.csv')
df['cyberbullying_type'].value_counts()

religion               7998
age                    7992
gender                 7973
ethnicity              7961
not_cyberbullying      7945
other_cyberbullying    7823
Name: cyberbullying_type, dtype: int64

**There is not much imbalance between different cyberbulling type. other_cyberbulling will be removed since it may cause a confusion for the models with other cyberbullying class.**

In [4]:
df.drop(df[df['cyberbullying_type'] == 'other_cyberbullying'].index, inplace = True)
df['cyberbullying_type'].value_counts()

religion             7998
age                  7992
gender               7973
ethnicity            7961
not_cyberbullying    7945
Name: cyberbullying_type, dtype: int64

# 1. Data Preprocessing

In [5]:
# Renaming Categories
df = df.rename(columns={'tweet_text': 'text', 'cyberbullying_type': 'sentiment'})

In [6]:
# Checking 10 samples
df.sample(10)

,text,sentiment
42145,Family? And I haven't had a Guinness in years....,ethnicity
46034,"Somalis should only date/marry other Somalis, ...",ethnicity
38540,they gonna bully this nigga at school and call...,age
17940,@G_Abdulazeez Stupid comment. ISIS is sponsore...,religion
44842,@AntiDARKSKINNED: Don't start with that I'm br...,ethnicity
18423,@loveconcursall This is an ISIS humanitarian c...,religion
14039,When do we get serious about impeaching Trump?...,gender
34681,"Was almost uncomfortable viewing man, like a s...",age
2240,Colin is back! #MKR,not_cyberbullying
20305,@buttercupashby @MaDaSaHaTtEr_17 Mohammed ran ...,religion


In [7]:
df = df.dropna()
df.head()

,text,sentiment
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [8]:
df.sentiment.unique()

array(['not_cyberbullying', 'gender', 'religion', 'age', 'ethnicity'],
      dtype=object)

In [9]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
y = LE.fit_transform(df["sentiment"])

In [10]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('omw-1.4')
sw_eng = stopwords.words('english')
def clean_review(review):
    '''
    Input:
        review: a string containing a review.
    Output:
        review_cleaned: a processed review. 

    '''
    review_in_lowercase = review.lower()
    no_punctuation = review_in_lowercase.translate(review_in_lowercase.maketrans('', '', string.punctuation))
    no_url = re.sub(r'https?:\/\/.*[\r\n]*','', no_punctuation)
    review_tokens = word_tokenize(no_url)
    no_stopwords_tokens = [token for token in review_tokens if not token in sw_eng]
    porter = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    review_cleaned = ''
    
    for each in no_stopwords_tokens:
        review_cleaned = review_cleaned + lemmatizer.lemmatize(each) + " "
    
    return review_cleaned

df['text'] = df['text'].apply(lambda x : clean_review(x))

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


# 2. Modeling

<b>Modeling to Predict Multi-ClassText Classification for types of Cyber Bullying </b>

Post preprocessing and sampling, the documents will tokenized and converted to an appropriate vector format for model consumption using Bag of Words (countvectorizer)

In [11]:
# TFIDF vectorizer on 
cv = CountVectorizer()

# fit and transform on dataset
X = cv.fit_transform(df.text).toarray()

In [ ]:
len(df)

39869

In [ ]:
len(X)

39869

In [ ]:
len(y)

39869

In [12]:
# Divide data into training and test (80%, 20% respectively)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2);

In [13]:
print ("Training Data Shape ")
print(len(x_train))
print(len(y_train))
print ("Test Data Shape ")
print(len(x_test))
print(len(y_test))

Training Data Shape 
31895
31895
Test Data Shape 
7974
7974


In [14]:
# Padding sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_size = len(max(x_train,key=lambda x_train:len(x_train)))
x_train = pad_sequences(x_train, maxlen=max_size, padding='post')

The CNN and GCN model will be experimented with to accomplish text classification. Below is the CNN implementation with different regularization and dropout layers

In [15]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Convolution1D
from keras.optimizers import SGD 
from tensorflow.keras.utils import to_categorical
tf.random.set_seed(31)

In [16]:
# X = np.expand_dims(x_train, axis=-1)

model = Sequential()

model.add(layers.Embedding(len(X), 100, input_length=max_size))

model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, activation = 'tanh')))

model.add(layers.Flatten())    

model.add(layers.Dense(64))

model.add(layers.Dropout(0.5))

model.add(layers.Dense(units=5, activation='sigmoid'))

model.compile(loss= 'categorical_crossentropy', optimizer='adam', metrics=['AUC', 'accuracy'])

In [17]:
history = model.fit(x_train, to_categorical(y_train, 5), batch_size = 64, epochs = 10,  verbose = 1, shuffle = True, validation_split=0.1);

Epoch 1/10
449/449 [==============================] - 1862s 4s/step - loss: 1.6114 - auc: 0.4993 - accuracy: 0.1991 - val_loss: 1.6098 - val_auc: 0.5000 - val_accuracy: 0.1994
Epoch 2/10
449/449 [==============================] - 1848s 4s/step - loss: 1.6106 - auc: 0.4970 - accuracy: 0.1969 - val_loss: 1.6099 - val_auc: 0.4993 - val_accuracy: 0.2009
Epoch 3/10
449/449 [==============================] - 1855s 4s/step - loss: 1.6103 - auc: 0.4981 - accuracy: 0.2001 - val_loss: 1.6102 - val_auc: 0.4999 - val_accuracy: 0.2009
Epoch 4/10
449/449 [==============================] - 1857s 4s/step - loss: 1.6103 - auc: 0.4987 - accuracy: 0.1974 - val_loss: 1.6096 - val_auc: 0.4991 - val_accuracy: 0.2009
Epoch 5/10
449/449 [==============================] - 1866s 4s/step - loss: 1.6102 - auc: 0.4988 - accuracy: 0.1986 - val_loss: 1.6097 - val_auc: 0.5024 - val_accuracy: 0.2031
Epoch 6/10
449/449 [==============================] - 1858s 4s/step - loss: 1.6101 - auc: 0.4988 - accuracy: 0.1994 - va